In [ ]:
import numpy as np
import time
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from scipy.spatial.transform import Rotation as R
import asyncio

In [ ]:
# Дані: профіль кутових швидкостей (рад/с)
omega = np.array([0.0, 0.08, 0.06])
dt = 0.1
T = 500
steps = int(T / dt)

# Метод кватерніонів
def integrate_quaternion(omega, dt, steps):
    q = np.array([1.0, 0.0, 0.0, 0.0])
    pos = np.array([0.0, 0.0, 0.0])
    pos_log = []
    v_body = np.array([10.0, 0.0, 0.0])

    for _ in range(steps):
        omega_quat = np.array([0.0, *omega])
        dq = 0.5 * quat_mult(q, omega_quat)
        q = q + dq * dt
        q = q / np.linalg.norm(q)

        r_mat = quaternion_to_matrix(q)
        vel = r_mat @ v_body
        pos = pos + vel * dt
        pos_log.append(pos.copy())

    return np.array(pos_log)

def quat_mult(q, p):
    w0, x0, y0, z0 = q
    w1, x1, y1, z1 = p
    return np.array([
        w0*w1 - x0*x1 - y0*y1 - z0*z1,
        w0*x1 + x0*w1 + y0*z1 - z0*y1,
        w0*y1 - x0*z1 + y0*w1 + z0*x1,
        w0*z1 + x0*y1 - y0*x1 + z0*w1
    ])

def quaternion_to_matrix(q):
    q0, q1, q2, q3 = q
    return np.array([
        [1-2*(q2**2 + q3**2), 2*(q1*q2 - q0*q3), 2*(q1*q3 + q0*q2)],
        [2*(q1*q2 + q0*q3), 1-2*(q1**2 + q3**2), 2*(q2*q3 - q0*q1)],
        [2*(q1*q3 - q0*q2), 2*(q2*q3 + q0*q1), 1-2*(q1**2 + q2**2)]
    ])

# Кути Ейлера
def integrate_euler(omega, dt, steps):
    r_total = R.identity()
    pos = np.array([0.0, 0.0, 0.0])
    pos_log = []
    v_body = np.array([10.0, 0.0, 0.0])

    d_r = R.from_euler('xyz',omega * dt)

    for _ in range(steps):
        r_total = r_total * d_r
        vel = r_total.apply(v_body)
        pos = pos + vel * dt
        pos_log.append(pos.copy())

    return np.array(pos_log)

# Матриці обертання
def integrate_matrix(omega, dt, steps):
    r_mat = np.eye(3)
    pos = np.array([0.0, 0.0, 0.0])
    pos_log = []
    v_body = np.array([10.0, 0.0, 0.0])

    for _ in range(steps):
        omega_mat = np.array([
            [0, -omega[2], omega[1]],
            [omega[2], 0, -omega[0]],
            [-omega[1], omega[0], 0]
        ])
        r_mat = r_mat @ (np.eye(3) + omega_mat * dt)
        vel = r_mat @ v_body
        pos = pos + vel * dt
        pos_log.append(pos.copy())

    return np.array(pos_log)

# Обчислення
start = time.time()
pos_quat = integrate_quaternion(omega, dt, steps)
t_q = time.time() - start

start = time.time()
pos_euler = integrate_euler(omega, dt, steps)
t_e = time.time() - start

start = time.time()
pos_matrix = integrate_matrix(omega, dt, steps)
t_m = time.time() - start

print("Час обчислення:")
print(f"  Кватерніони: {t_q:.5f} c")
print(f"  Кути Ейлера: {t_e:.5f} c")
print(f"  Матриці:     {t_m:.5f} c")

# Графік
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=pos_quat[:,0], y=pos_quat[:,1], z=pos_quat[:,2],
                           mode='lines', name="Кватерніони", line=dict(color='blue')))
fig.add_trace(go.Scatter3d(x=pos_euler[:,0], y=pos_euler[:,1], z=pos_euler[:,2],
                           mode='lines', name="Кути Ейлера", line=dict(color='green')))
fig.add_trace(go.Scatter3d(x=pos_matrix[:,0], y=pos_matrix[:,1], z=pos_matrix[:,2],
                           mode='lines', name="Матриці обертання", line=dict(color='red')))
fig.update_layout(
    scene=dict(
        xaxis_title='X (м)',
        yaxis_title='Y (м)',
        zaxis_title='Z (м)',
    ),
    title="Порівняння траєкторій літальних апаратів",
    width=900, height=600
)

fig.show()


In [ ]:
# Дані: профіль кутових швидкостей (рад/с)
omega = np.array([0, 0.08, 0.06])
dt = 0.1
T = 500
steps = int(T / dt)

# Обчислення траєкторій
pos_quat = integrate_quaternion(omega, dt, steps)
pos_euler = integrate_euler(omega, dt, steps)
pos_matrix = integrate_matrix(omega, dt, steps)

# Графік
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=pos_quat[:,0], y=pos_quat[:,1], z=pos_quat[:,2],
                           mode='lines', name="Кватерніони", line=dict(color='blue')))
fig.add_trace(go.Scatter3d(x=pos_euler[:,0], y=pos_euler[:,1], z=pos_euler[:,2],
                           mode='lines', name="Кути Ейлера", line=dict(color='green')))
fig.add_trace(go.Scatter3d(x=pos_matrix[:,0], y=pos_matrix[:,1], z=pos_matrix[:,2],
                           mode='lines', name="Матриці обертання", line=dict(color='red')))

# Кнопки для вибору методу траєкторії
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="Всі",
                     method="update",
                     args=[{"visible": [True, True, True]},
                           {"title": "Всі траєкторії"}]),
                dict(label="Кватерніони",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Траєкторія: Кватерніони"}]),
                dict(label="Кути Ейлера",
                     method="update",
                     args=[{"visible": [False, True, False]},
                           {"title": "Траєкторія: Кути Ейлера"}]),
                dict(label="Матриці обертання",
                     method="update",
                     args=[{"visible": [False, False, True]},
                           {"title": "Траєкторія: Матриці обертання"}]),
            ]),
            direction="down",
            showactive=True,
            x=0,
            y=0.5,
            yanchor="bottom"
        ),
    ]
)

fig.update_layout(
    scene=dict(
        xaxis_title='X (м)',
        yaxis_title='Y (м)',
        zaxis_title='Z (м)',
    ),
    title="Вибір траєкторій обертання",
    width=800, height=550
)

fig.show()


In [ ]:
# Дані
omega = np.array([0, 0, 0.07])
dt = 0.1
v_body = np.array([10.0, 0.0, 0.0])

# Стан симуляції
step = 0
max_steps = 500
running = False

# Початкові стани
q = np.array([1.0, 0.0, 0.0, 0.0])
R_mat = np.eye(3)
pos_q = np.array([0.0, 0.0, 0.0])
pos_e = np.array([0.0, 0.0, 0.0])
pos_m = np.array([0.0, 0.0, 0.0])
R_total = R.identity()
log_q = []
log_e = []
log_m = []

dR = R.from_euler('xyz', omega * dt)

# Крок моделювання
def step_simulation():
    global step, q, R_mat, pos_q, pos_e, pos_m, R_total

    if step >= max_steps:
        return False

    # Кватерніони
    omega_quat = np.array([0.0, *omega])
    dq = 0.5 * quat_mult(q, omega_quat)
    q = q + dq * dt
    q = q / np.linalg.norm(q)
    vel_q = quaternion_to_matrix(q) @ v_body
    pos_q[:] += vel_q * dt
    log_q.append(pos_q.copy())

    # Ейлер
    R_total = R_total * dR
    vel_e = R_total.apply(v_body)
    pos_e[:] += vel_e * dt
    log_e.append(pos_e.copy())

    # Матриця
    omega_mat = np.array([
        [0, -omega[2], omega[1]],
        [omega[2], 0, -omega[0]],
        [-omega[1], omega[0], 0]
    ])
    R_mat = R_mat @ (np.eye(3) + omega_mat * dt)
    vel_m = R_mat @ v_body
    pos_m[:] += vel_m * dt
    log_m.append(pos_m.copy())

    step += 1
    return True

fig = go.FigureWidget()
fig.add_scatter3d(mode='lines', name='Кватерніони', line=dict(color='blue'))
fig.add_scatter3d(mode='lines', name='Кути Ейлера', line=dict(color='green'))
fig.add_scatter3d(mode='lines', name='Матриці', line=dict(color='red'))
fig.update_layout(width=800, height=600)

def update_plot():
    with fig.batch_update():
        fig.data[0].x = [p[0] for p in log_q]
        fig.data[0].y = [p[1] for p in log_q]
        fig.data[0].z = [p[2] for p in log_q]

        fig.data[1].x = [p[0] for p in log_e]
        fig.data[1].y = [p[1] for p in log_e]
        fig.data[1].z = [p[2] for p in log_e]

        fig.data[2].x = [p[0] for p in log_m]
        fig.data[2].y = [p[1] for p in log_m]
        fig.data[2].z = [p[2] for p in log_m]

start_button = widgets.Button(description="Start")
stop_button = widgets.Button(description="Stop")

def start_clicked(b):
    global running
    if not running:
        running = True
        asyncio.ensure_future(run_steps())

def stop_clicked(b):
    global running
    running = False

start_button.on_click(start_clicked)
stop_button.on_click(stop_clicked)

# Цикл моделювання
async def run_steps():
    global running
    while running and step < max_steps:
        step_simulation()
        update_plot()
    print("Симуляція завершена або зупинена.")

display(start_button, stop_button, fig)


In [ ]:
# Дані
omega = np.array([0, 0, 0.07])
dt = 0.1
v_body = np.array([2.0, 0.0, 0.0])
max_steps = 500

# Логи для всіх методів
log_q = []
log_e = []
log_m = []

# Початкові стани
q = np.array([1.0, 0.0, 0.0, 0.0])
R_mat = np.eye(3)
pos_q = np.zeros(3)
pos_e = np.zeros(3)
pos_m = np.zeros(3)
R_total = R.identity()
dR = R.from_euler('xyz', omega * dt)

# Моделювання
for step in range(max_steps):
    # Кватерніони
    omega_quat = np.array([0.0, *omega])
    dq = 0.5 * quat_mult(q, omega_quat)
    q = q + dq * dt
    q = q / np.linalg.norm(q)
    vel_q = quaternion_to_matrix(q) @ v_body
    pos_q[:] += vel_q * dt
    log_q.append((pos_q.copy(), [q[1], q[2], q[3], q[0]]))

    # Ейлер
    R_total = R_total * dR
    pos_e += R_total.apply(v_body) * dt
    log_e.append((pos_e.copy(), R_total.as_quat()))

    # Матриці
    Omega = np.array([
        [0, -omega[2], omega[1]],
        [omega[2], 0, -omega[0]],
        [-omega[1], omega[0], 0]
    ])
    R_mat = R_mat @ (np.eye(3) + Omega * dt)
    pos_m += R_mat @ v_body * dt
    rot_m = R.from_matrix(R_mat)
    log_m.append((pos_m.copy(), rot_m.as_quat()))

# Анімація
frames = []
N = min(len(log_q), len(log_e), len(log_m))

for i in range(0, N, 5):
    pos_q_i, q_q = log_q[i]
    pos_e_i, q_e = log_e[i]
    pos_m_i, q_m = log_m[i]

    def quat_to_dir(q):
        q1, q2, q3, q0 = q
        R_ = np.array([
            [1 - 2*(q2**2 + q3**2), 2*(q1*q2 - q0*q3), 2*(q1*q3 + q0*q2)],
            [2*(q1*q2 + q0*q3), 1 - 2*(q1**2 + q3**2), 2*(q2*q3 - q0*q1)],
            [2*(q1*q3 - q0*q2), 2*(q2*q3 + q0*q1), 1 - 2*(q1**2 + q2**2)]
        ])
        return R_ @ np.array([1, 0, 0])

    dir_q = quat_to_dir(q_q)
    dir_e = quat_to_dir(q_e)
    dir_m = quat_to_dir(q_m)

    frame = go.Frame(
        data=[
            go.Cone(x=[pos_q_i[0]], y=[pos_q_i[1]], z=[pos_q_i[2]],
                    u=[dir_q[0]], v=[dir_q[1]], w=[dir_q[2]],
                    sizemode="absolute", sizeref=5, anchor="tip",
                    colorscale="Blues", showscale=False, name="Кватерніон"),
            go.Cone(x=[pos_e_i[0]], y=[pos_e_i[1]], z=[pos_e_i[2]],
                    u=[dir_e[0]], v=[dir_e[1]], w=[dir_e[2]],
                    sizemode="absolute", sizeref=5, anchor="tip",
                    colorscale="Greens", showscale=False, name="Ейлер"),
            go.Cone(x=[pos_m_i[0]], y=[pos_m_i[1]], z=[pos_m_i[2]],
                    u=[dir_m[0]], v=[dir_m[1]], w=[dir_m[2]],
                    sizemode="absolute", sizeref=5, anchor="tip",
                    colorscale="Reds", showscale=False, name="Матриця"),
            go.Scatter3d(x=[p[0][0] for p in log_q[:i+1]],
                         y=[p[0][1] for p in log_q[:i+1]],
                         z=[p[0][2] for p in log_q[:i+1]],
                         mode='lines', line=dict(color='blue', width=2), name="Кватерніон"),
            go.Scatter3d(x=[p[0][0] for p in log_e[:i+1]],
                         y=[p[0][1] for p in log_e[:i+1]],
                         z=[p[0][2] for p in log_e[:i+1]],
                         mode='lines', line=dict(color='green', width=2), name="Ейлер"),
            go.Scatter3d(x=[p[0][0] for p in log_m[:i+1]],
                         y=[p[0][1] for p in log_m[:i+1]],
                         z=[p[0][2] for p in log_m[:i+1]],
                         mode='lines', line=dict(color='red', width=2), name="Матриця"),
        ]
    )
    frames.append(frame)

init_data = frames[0].data
fig = go.Figure(
    data=init_data,
    layout=go.Layout(
        updatemenus=[dict(
            type="buttons",
            buttons=[
                dict(label="Play", method="animate",
                     args=[None, {"frame": {"duration": 50, "redraw": True},
                                  "fromcurrent": True, "mode": "immediate"}]),
                dict(label="Pause", method="animate",
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate"}])
            ]
        )]
    ),
    frames=frames
)

fig.update_layout(title="Анімація руху літаків в 3D (Кватерніони, Ейлер, Матриці)")
fig.show()
